# Profile Generation

After creation of profile lines cross cutting the river at n intervals and intersecting with the buffered cross cuts we will have all the x,y,z points in the profile. We can then plot these points and interpolate the bottom of the slough. Add in the tidal elevation limits as well. 

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatch
import numpy as np
from scipy.interpolate import CubicSpline,interp1d
from shapely.geometry import Point,Polygon,LineString
from descartes import PolygonPatch
import math
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
plt.rcParams['font.family'] = 'Arial'

In [ ]:
def profileDistance(x,y,lx0,ly0,lx1,ly1):
    # the distance a point resides along the profile line
    # this is the projection of the point onto the line aka 
    # the dot product of the two vectors divided by the unit vector of the line. 
    # after convert eveything to be relative to the first vector / making it the new origin.
    v2 = np.array((lx1,ly1)) - np.array((lx0,ly0))
    v1 = np.array((x,y)) - np.array((lx0,ly0))
    d = np.dot(v1,v2) / np.linalg.norm(v2)
    return d


def interpolateTopography(dvals,zvals):
    # We create an interpolated profile using the distance along the x-section and the elevation values. 
    # the dvals, zvals must be sorted by dval and strictly dvals increasing. 
    cs = interp1d(dvals, zvals)
    xs = np.linspace(dvals[0],dvals[-1], 500)
    zs = cs(xs)
    
    return xs,zs

def makeGeometry(xs,zs,minz):
    
    # make a shapely polygon of the geometry. 
    
    #minz = np.min(zs)
    pointsProfile = [Point(x,z) for x,z in zip(xs,zs)]
    pointsBottom = [Point(x,minz) for x in np.flip(xs)]
    pointsFinal =  pointsProfile + pointsBottom 
    poly = Polygon([[p.x, p.y] for p in pointsFinal])
    
    return poly

def makeTidalGeometry(xs,low,high):
    
    pointshigh = [Point(x,high) for x in xs]
    pointslow = [Point(x,low) for x in np.flip(xs)]
    pointsFinal =  pointshigh + pointslow
    poly = Polygon([[p.x, p.y] for p in pointsFinal])
    
    return poly


def makeTidalRangeBars(x,y,low,high): 
    
    vertlineAbove = LineString([Point(x,y),Point(x,high)])
    vertlineBelow = LineString([Point(x,y),Point(x,low)])
    crossbarAbove = LineString([Point(x-0.03*x,high),Point(x+0.03*x,high)])
    crossbarBelow = LineString([Point(x-0.03*x,low),Point(x+0.03*x,low)])
    
    return [vertlineAbove,vertlineBelow,crossbarAbove,crossbarBelow]


def makeProfile(xs,zs,tideLow,tideHigh,props,geo,tidalgeo,watergeo):
    """
    props should be a dictionary with the following:
    {
    'title':"Profile 1",
    'titleSize':32,
    'ylabel' : 'Depth (Feet)'
    'xlabel' : 'Distance Along Profile Line (Map Units)'
    'axeslabelsize' : 12,
    'tidalrangesize': 14,
    'figsize': (10,10),
    "figYlims":(-60,5)
        }
    
    """
    
    fig,axes = plt.subplots(1,1,figsize=props['figsize'])
    # plot the upper and lower tide zones
    
    axes.set_xlabel(props['xlabel'],size=props['axeslabelsize'])
    axes.set_ylabel(props['ylabel'],size=props['axeslabelsize'])
    
    fig.suptitle(props['title'],size=props['titleSize'],y=0.94,fontweight='bold')
    axes.add_patch(PolygonPatch(watergeo,facecolor='#0E2F52', linestyle='--',zorder=2,alpha=0.7))
    axes.add_patch(PolygonPatch(tidalgeo,facecolor='#2886EB', edgecolor='#202020',linestyle='--',zorder=2,alpha=0.7))
    axes.add_patch(PolygonPatch(geo,facecolor='#fff4d6', edgecolor='#ab9a00',hatch="...",zorder=2))
    axes.set_xlim(xs[0],xs[-1])
    axes.set_ylim(np.min(zs))
    tidalx = (xs[0]+xs[-1])/5
    tidaly = (tideHigh+tideLow)/2
    axes.text(x=tidalx, y =tidaly ,s='Tidal Range',style='italic',size=props['tidalrangesize'],color='black',
              zorder=10,bbox=(dict(facecolor='white',edgecolor='black')))
    bars = makeTidalRangeBars(tidalx+0.3*tidalx,tidaly,tideLow,tideHigh)
    axes.set_ylim(props['figYlims'])
    axes.grid(b=True,zorder=10,ls='--',which='both')
    axes.yaxis.set_minor_locator(MultipleLocator(2))
    axes.xaxis.set_minor_locator(MultipleLocator(10))
    axes.tick_params(which='major', length=10, labelsize=16)
    axes.tick_params(which='minor', length=5)

    for b in bars:
        x, y = b.xy
        axes.plot(x,y,color='black',linestyle='-')
        
#     axes.set_aspect(5, adjustable=None, anchor=None, share=False)
    return fig,axes

def generateProfiles(lineShape,pointsShape,props):
    """
    Takes the input geopandas gdf lineStrings broken out in attribute table by an identifier and the associated pointsShape file with 
    the same attribute then generates a profile for each of the points groupings. 
    
    props = {
        lineIDcol = 'IDCol',
        tiderange = (0.7,7),
        figprops =[ {
                        'title' : 'Test Station Profile Number X.',
                        'titleSize' : 16,
                        'ylabel' : 'Depth or Elevation (Feet)',
                        'xlabel' : 'Distance Along Profile Line (Map Units)',
                        'axeslabelsize' : 14,
                        'tidalrangesize': 14,
                        'figsize': (10,10)
                    }]
    }
    """
    # loop over each of the lines
    lines = lineShape[props['lineIDcol']].unique()
    
    for i,line in enumerate(lines): 
        linei = lineShape.loc[lineshape[props['lineIDcol']] == line]
        pointsi = pointsShape.loc[pointsShapeprops['lineIDcol'] == line]
        
        line0 = line.geometry[0].coords[0]
        line1 = line.geometry[0].coords[1]
       
        pointsShape['distance'] = [profileDistance(x,y,line0[0],line0[1],line1[0],line1[1]) for x,y in zip(points['long'],points['lat'])]

        pointsShape = pointsShape.sort_values(by='distance',ascending=True)
        xs,zs = interpolateTopography(pointsShape['distance'].values, pointsShape['depth'].values)

        geo = makeGeometry(xs,zs*-1)
        tidalgeo = makeTidalGeometry(xs,props['tiderange'][0],props['tiderange'][1])
        
        figure,ax = makeProfile(xs,zs*-1, 0.7,-7,props['figprops'][i],geo,tidalgeo)
        
        figure.savefig(props['figprops'][i]['title']+".png")
        
def zerosLeading(number):
    l = len(number)
    if l==1:
        return("0{}".format(number))
    else:
        return("{}".format(number))

def stationNum(distance,slough):
    if distance == -1:
        return "T94 Shoring Pile"
    else:
        txt1 = str(int(math.floor(distance/100)))
        txt2 = str(int(distance % 100))   
        ftext = "{} Station {} + {}".format(slough,txt1,zerosLeading(txt2))
   
    return ftext

# Example 

In [ ]:
lineinit = gpd.read_file("Data/Final/ProfileLinesFinal.shp")
pointsinit = gpd.read_file("Data/Final/ProfilePoints.shp")
pointsinit['Stationtext'] = [stationNum(d,s) for d,s in zip(pointsinit['Distance'], pointsinit['SloughName'])]
lineinit['Stationtext'] = [stationNum(d,s) for d,s in zip(lineinit['Distance'], lineinit['SloughName'])]

lower = pointsinit['Elevation'].min()
upper = pointsinit['Elevation'].max()
lines = lineinit['Stationtext'].unique()

figs = {}

for l in lines:
    points = pointsinit.loc[pointsinit['Stationtext'] == l]
    line = lineinit.loc[lineinit['Stationtext'] == l]
    for i in range(2000):
        try:
            line0 = line.geometry.iloc[i].coords[0]
            line1 = line.geometry.iloc[i].coords[1]
        except:
            pass

    print(line0)
    points['distance'] = [profileDistance(x,y,line0[0],line0[1],line1[0],line1[1]) for x,y in zip(points['X'],points['Y'])]
    points = points.sort_values(by='distance',ascending=True)
    xs,zs = interpolateTopography(points['distance'].values, points['Elevation'].values)
    geo = makeGeometry(xs,zs,lower - 5)
    tidalgeo = makeTidalGeometry(xs,-1.5,7.5)
    watergeo = makeTidalGeometry(xs,-50,-1.5)
    props = {
        'title' :l,
        'titleSize' : 30,
        'ylabel' : 'Elevation (Feet - MLLW)',
        'xlabel' : 'Distance Along Profile Line (Feet)',
        'axeslabelsize' : 24,
        'tidalrangesize': 14,
        'figsize':(np.max(xs)/20,7.5),
        "figYlims": (lower-5,upper+5)
    }
    figureax = makeProfile(xs,zs, -1.5,7.5,props,geo,tidalgeo,watergeo)
    fig, ax = figureax[0],figureax[1]
    figs[l] = figureax
    
    fig.savefig("All Profiles\{}.png".format(l),dpi=300,bbox_inches='tight')
    
    del fig,ax